<a href="https://colab.research.google.com/github/JdVaru/Machine-Learning-Exercises/blob/master/Property_Inspection_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0 Install required packages 

In [1]:
!pip install kaggle
!pip install numpy==1.16.3
!pip install catboost

     |████████████████████████████████| 60.6MB 550kB/s 


# Step 1 Import required packages

In [0]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge
from sklearn import preprocessing

# Step 2 Import data 

In [3]:
# colab file access 
from google.colab import files

# Retrieve data
uploaded = files.upload()

# move jsaon where API expect data
!mkdir -p ~/.kaggle && mv kaggle.json ~/.kaggle && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [4]:
# download data from kaggle
!kaggle competitions download -c liberty-mutual-group-property-inspection-prediction

  0% 0.00/56.6k [00:00<?, ?B/s]
100% 56.6k/56.6k [00:00<00:00, 47.7MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 63.3MB/s]
  0% 0.00/930k [00:00<?, ?B/s]
100% 930k/930k [00:00<00:00, 130MB/s]


In [5]:
# unzip dowloaded data
!unzip train.csv.zip
!unzip test.csv.zip
!unzip sample_submission.csv.zip


Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   


In [0]:
#load train and test 
train = pd.read_csv('train.csv', index_col=0)
test  = pd.read_csv('test.csv', index_col=0)

In [3]:
train.head()

,Hazard,T1_V1,T1_V2,T1_V3,T1_V4,T1_V5,T1_V6,T1_V7,T1_V8,T1_V9,T1_V10,T1_V11,T1_V12,T1_V13,T1_V14,T1_V15,T1_V16,T1_V17,T2_V1,T2_V2,T2_V3,T2_V4,T2_V5,T2_V6,T2_V7,T2_V8,T2_V9,T2_V10,T2_V11,T2_V12,T2_V13,T2_V14,T2_V15
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,15,3,2,N,B,N,B,B,D,7,B,B,15,1,A,B,N,36,11,N,10,B,2,37,1,11,6,Y,N,E,2,2
2,4,16,14,5,H,B,N,B,B,C,12,B,B,10,3,A,B,Y,78,10,Y,17,C,2,22,1,18,5,Y,Y,E,2,1
3,1,10,10,5,N,K,N,B,B,E,12,H,B,15,1,A,R,Y,71,21,Y,13,C,6,37,2,14,6,Y,Y,E,6,1
4,1,18,18,5,N,K,N,B,B,E,3,H,B,15,1,A,R,N,71,13,N,15,A,2,25,1,1,6,Y,N,C,2,6
5,1,13,19,5,N,H,N,B,B,E,7,H,B,10,1,A,J,N,75,10,Y,11,B,1,22,1,2,7,N,N,E,1,1


In [4]:
test.head()

,T1_V1,T1_V2,T1_V3,T1_V4,T1_V5,T1_V6,T1_V7,T1_V8,T1_V9,T1_V10,T1_V11,T1_V12,T1_V13,T1_V14,T1_V15,T1_V16,T1_V17,T2_V1,T2_V2,T2_V3,T2_V4,T2_V5,T2_V6,T2_V7,T2_V8,T2_V9,T2_V10,T2_V11,T2_V12,T2_V13,T2_V14,T2_V15
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,2,13,4,C,A,Y,B,B,D,12,F,B,15,2,S,B,N,48,11,Y,7,A,2,28,1,22,6,Y,N,E,2,7
7,10,10,7,N,C,Y,B,B,D,8,L,B,15,1,A,I,N,14,10,Y,14,A,3,28,1,4,3,Y,N,E,5,8
8,9,20,4,N,H,Y,B,B,E,8,H,B,20,2,A,R,N,19,12,Y,6,B,2,22,1,1,7,N,N,C,6,1
9,11,18,2,N,H,Y,B,B,D,12,L,B,5,1,A,B,N,51,23,Y,10,A,4,40,1,20,6,Y,N,E,5,5
10,4,5,4,H,K,Y,B,B,E,8,I,B,15,3,A,K,Y,75,9,N,15,C,2,34,1,11,7,Y,Y,E,2,1


In [0]:
# Define Input and output data
y = train.Hazard
X = train.drop('Hazard', axis=1)

columns = X.columns
test_ind = test.index

In [0]:
# label encode the categorical variables
X = np.array(X)
test = np.array(test)

for i in range(X.shape[1]):
    if type(X[1,i]) is str:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X[:,i]) + list(test[:,i]))
        X[:,i] = lbl.transform(X[:,i])
        test[:,i] = lbl.transform(test[:,i])

In [7]:
# Null value validation
def null_val(in_var):
    X_tr = pd.DataFrame(in_var)
    for col in X_tr.columns:
        if X_tr[col].isnull().any():
            print(col)
            
print(null_val(X))
print(null_val(test))

None
None


In [0]:
# Normalize data using standard scaler
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

scaler.fit(test)
test_scaled = scaler.transform(test)

In [9]:
X_scaled

array([[ 1.02128809, -1.57418252, -0.68186527, ...,  0.77926141,
        -0.35801939, -0.48248866],
       [ 1.21479056,  0.18421888,  1.0429151 , ...,  0.77926141,
        -0.35801939, -0.80751064],
       [ 0.05377577, -0.45519981,  1.0429151 , ...,  0.77926141,
         2.81642852, -0.80751064],
       ...,
       [ 1.60179549, -0.93476383,  1.0429151 , ...,  0.77926141,
        -1.15163137, -0.80751064],
       [ 1.60179549,  0.6637829 ,  1.0429151 , ..., -0.48881621,
        -0.35801939,  0.81759926],
       [-0.91373656,  0.34407356, -0.10693848, ...,  0.77926141,
         2.02281654,  0.1675553 ]])

In [32]:
def gini(solution, submission):
    df = zip(solution, submission, range(len(solution)))
    df = sorted(df, key=lambda x: (x[1],-x[2]), reverse=True)
    rand = [float(i+1)/float(len(df)) for i in range(len(df))]
    totalPos = float(sum([x[0] for x in df]))
    cumPosFound = [df[0][0]]
    for i in range(1,len(df)):
        cumPosFound.append(cumPosFound[len(cumPosFound)-1] + df[i][0])
    Lorentz = [float(x)/totalPos for x in cumPosFound]
    Gini = [Lorentz[i]-rand[i] for i in range(len(df))]
    return sum(Gini)

def normalized_gini(solution, submission):
    normalized_gini = gini(solution, submission)/gini(solution, solution)
    return normalized_gini


print(normalized_gini(range(10)[::-1], [1]*10))

# test case 1
desired_y = np.array(pd.read_csv("train.csv")["Hazard"])
predicted_y = np.copy(desired_y)
predicted_y[:] = 1
print(normalized_gini(desired_y, predicted_y))
print(normalized_gini(desired_y[::-1], predicted_y))

## test case 2
predicted_y = [i + (i%30) for i in range(100)] 
desired_y = [i for i in range(100)]
print(normalized_gini(predicted_y, desired_y))

## test case 3
predicted_y = [i for i in range(100)] 
desired_y = [i + (i%30) for i in range(100)]
print(normalized_gini(predicted_y, desired_y))

1.0
-0.007730752488393131
0.007730752488393188
0.9626840508036416
0.9578157815781583


In [10]:
# Model 1: Xgboost
params = {}
params["objective"] = "reg:linear"
params["eta"] = 0.01
params["min_child_weight"] = 5
params["subsample"] = 0.8
params["scale_pos_weight"] = 1.0
params["silent"] = 1
params["max_depth"] = 7

plst = list(params.items())

#Using 5000 rows for early stopping. 
offset = 5000

num_rounds = 2000
xgtest = xgb.DMatrix(test_scaled)

#create a train and validation dmatrices 
xgtrain = xgb.DMatrix(X_scaled[offset:,:], label=y[offset:])
xgval = xgb.DMatrix(X_scaled[:offset,:], label=y[:offset])

#train using early stopping and predict
watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=5)
preds1 = model.predict(xgtest)


#reverse train and labels and use different 5k for early stopping. 
# this adds very little to the score but it is an option if you are concerned about using all the data. 
X_scaled_reverse = X[::-1,:]
y_reverse = y[::-1]

xgtrain = xgb.DMatrix(X_scaled_reverse[offset:,:], label=y_reverse[offset:])
xgval = xgb.DMatrix(X_scaled_reverse[:offset,:], label=y_reverse[:offset])

watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=5)
preds2 = model.predict(xgtest)

#combine predictions
#since the metric only cares about relative rank we don't need to average
preds = preds1 + preds2

#generate solution
preds = pd.DataFrame({"Id": test_ind, "Hazard": preds})
preds = preds.set_index('Id')
preds.to_csv('xgboost_benchmark.csv')

[0]	train-rmse:5.32545	val-rmse:5.26836
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 5 rounds.
[1]	train-rmse:5.29964	val-rmse:5.24332
[2]	train-rmse:5.27407	val-rmse:5.21843
[3]	train-rmse:5.24922	val-rmse:5.19408
[4]	train-rmse:5.22446	val-rmse:5.16983
[5]	train-rmse:5.20017	val-rmse:5.14608
[6]	train-rmse:5.17609	val-rmse:5.12244
[7]	train-rmse:5.15263	val-rmse:5.09958
[8]	train-rmse:5.12933	val-rmse:5.07689
[9]	train-rmse:5.10667	val-rmse:5.05479
[10]	train-rmse:5.08421	val-rmse:5.03274
[11]	train-rmse:5.06179	val-rmse:5.0111
[12]	train-rmse:5.04001	val-rmse:4.98982
[13]	train-rmse:5.01857	val-rmse:4.9688
[14]	train-rmse:4.99731	val-rmse:4.94808
[15]	train-rmse:4.97626	val-rmse:4.92767
[16]	train-rmse:4.95575	val-rmse:4.90769
[17]	train-rmse:4.9356	val-rmse:4.88807
[18]	train-rmse:4.91567	val-rmse:4.86855
[19]	train-rmse:4.89589	val-rmse:4.84959
[20]	train-rmse:4.87667	val-rmse:4.83108
[21]	train-r

In [0]:
# Model 2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score

from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier,RandomForestRegressor
from sklearn.svm import SVC,SVR

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [0]:
# Performing the train test split
y=y.values
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state = 0)

In [13]:
X_train.shape

(40799, 32)

In [0]:
from sklearn.metrics import mean_squared_error,r2_score
def validate(X_train, y_train,clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    std = np.std(y_test)
    print("Standard Deviation σ : {}".format(std))
    
    # Compute and print R^2 # it shoud be near to one
    print("R^2: {}".format(r2_score(y_test,y_pred)))
    # Compute and print RMSE # it should be smaller than standard deviation
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Root Mean Squared Error: {}".format(rmse))
    # Compute and print Gini    
    print("Gini score: {}".format(normalized_gini(y_pred, y_test)))

In [29]:
# Accuracy and ROC for LogisticRegression

clf1 = LinearRegression()
validate(X_train,y_train,clf1)

Standard Deviation σ : 4.093636226484545
R^2: 0.05736891331880767
Root Mean Squared Error: 3.9744782647936496
Gini score: 0.22627630029577414


In [0]:
# Accuracy and ROC for SVC

#clf2 = SVR(C = 6, kernel = 'rbf')
#validate(X_train,y_train,clf2)

In [33]:
# Accuracy and ROC for RabdomForest

clf3 = RandomForestRegressor(n_estimators = 10, random_state = 123,
                                       min_samples_leaf = 1, min_samples_split = 3, n_jobs = -1)
validate(X_scaled,y,clf3)

Standard Deviation σ : 4.093636226484545
R^2: 0.8000862237322445
Root Mean Squared Error: 1.8303351020877374
Gini score: 0.8791322317207982


In [0]:
xgt = xgb.DMatrix(X_test)
y_pred = model.predict(xgt)

In [37]:
std = np.std(y_test)
print("Standard Deviation σ : {}".format(std))

# Compute and print R^2 # it shoud be near to one
print("R^2: {}".format(r2_score(y_test,y_pred)))
# Compute and print RMSE # it should be smaller than standard deviation
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))
# Compute and print Gini    
print("Gini score: {}".format(normalized_gini(y_pred, y_test)))

Standard Deviation σ : 4.093636226484545
R^2: -0.10302314278554636
Root Mean Squared Error: 4.29933770555806
Gini score: 0.17859396814472164
